In [1]:
import numpy as np
import sys
import bisect
sys.path.append('../')
from utils.sparse_vector import *
from utils.vector_polynomial_isomorphism import *
from utils.minor_flattening import *

This is an example with $\hat{\sigma_{2}}(\mathbb{P}^{1} \times \mathbb{P}^{1} \times \mathbb{P}^{1} \times \mathbb{P}^{1})$.

In [2]:
# Let us initialize constants and vector spaces.

N = 2*2*2*2
n = [2,2,2,2]
d = 3
indeterminates = form_indeterminates(n)
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)

Defining x_0000, x_0001, x_0010, x_0011, x_0100, x_0101, x_0110, x_0111, x_1000, x_1001, x_1010, x_1011, x_1100, x_1101, x_1110, x_1111


In [3]:
def E_action(a, b, grad_fl):
    sum = -1*indeterminates[b]*grad_fl[a]
    return R(sum)

In [4]:
f = get_flattening_minors(R, n, d)
monomial_to_index = {}

# create (monomial => index) mapping for the monomials in all f_i
for fi in f:
    for mon in fi.monomials():
        if mon not in monomial_to_index:
            monomial_to_index[mon] = len(monomial_to_index)

dim_V_small = len(monomial_to_index)
V_small = VectorSpace(QQ, dim_V_small, sparse=True)

# add in monomials that are in span{monomials of Xf_i} to the (monomial => index) mapping
for fl in f:
    grad_fl = fl.gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl_ij = E_action(i,j,grad_fl)
            for mon in E_action_fl_ij.monomials():
                if mon not in monomial_to_index:
                    monomial_to_index[mon] = len(monomial_to_index)

# convert f_i into vectors inside V_small
f_vec = []
for fi in f:
    fi_vec = [0]*dim_V_small
    for mon in fi.monomials():
        idx = monomial_to_index[mon]
        fi_vec[idx] = fi.monomial_coefficient(mon)
    f_vec.append(V_small(fi_vec))

In [5]:
# create W = span{f_i} inside V_small. W_perp is its orthogonal complement inside this small vector space
W = V_small.subspace(f_vec)
W_perp = W.basis_matrix().right_kernel_matrix(algorithm='default', basis='computed')

p = W.dimension()
q = W_perp.nrows()

print('p=', p, 'q=', q, 'dim_V_small=', dim_V_small)

p= 32 q= 112 dim_V_small= 144


In [6]:
# create sparse basis vectors for orthogonal complement in V_small
g_sparse = [sparsify(W_perp[k]) for k in range(q)]

# add g_i's to include into {monomials of fi}U{monomials of Xfi} by adding standard basis vectors in coordinates outside V_small
for i in range(dim_V_small, len(monomial_to_index)):
    v = [(i, 1)]
    g_sparse.append(v)

q = len(g_sparse)

In [7]:
def polynomial_to_sparsevector(p):
    ans = []
    for mon in p.monomials():   
        bisect.insort(ans, (monomial_to_index[mon], p.monomial_coefficient(mon)))
    return ans

E_action_fl = [[0 for j in range(N)] for i in range(N)]

M_f = zero_matrix(QQ, p*q, N^2, sparse = True)
for l in range(p):
    grad_fl = f[l].gradient()
    for i in range(N):
        for j in range(N):
            E_action_fl[i][j] = polynomial_to_sparsevector(E_action(i,j,grad_fl))
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(q*l) + k,(N*i) + j] = sparse_inner_product(E_action_fl[i][j], g_sparse[k])

In [8]:
rank = M_f.rank()

print('rank(M_f) =', rank)
print('dim(ker(M_f)) =', glN.dimension() - rank)
print('expected:', sum([ni^2-1 for ni in n]) + 1)

rank(M_f) = 243
dim(ker(M_f)) = 13
expected: 13
